# Deployment of the while pipeline with WatsonML

#### Tjark Petersen, May 2020

This code would not work because Watson ML does only support the standard pipelines and models, my custom transformers (feature engineering) are not supported. Therefore, use the Flask App for scoring.

In [8]:
import pyspark
import pandas as pd
from joblib import load
from watson_machine_learning_client import WatsonMachineLearningAPIClient
from sklearn.model_selection import train_test_split

In [ ]:
wml_credentials={
  "url": "https://eu-gb.ml.cloud.ibm.com",
  "apikey": "xxx",
  "username": "Service credentials-1",
  "instance_id": "xxx"
}

client = WatsonMachineLearningAPIClient(wml_credentials)

In [5]:
final_pipeline = load('revenue_prediction_final_pipeline.joblib') 

In [9]:
# example input
df = pd.read_csv('online_shoppers_intention.csv')
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Tjark Petersen", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "tpetersen.uk@gmail.com", 
               client.repository.ModelMetaNames.NAME: "shopping_behaviour_final_model"}

model_artifact=client.repository.store_model(final_pipeline, meta_props=model_props, training_data=X_train, training_target=y_train)

In [ ]:
#Get model UID
published_model_uid = client.repository.get_model_uid(model_artifact)
#Deploy the model
created_deployment = client.deployments.create(published_model_uid, name="shopping_behaviour_deployment")

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
print(scoring_endpoint)
# example:
# https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/f45142b5-b178-444d-b30b-494230b95138/deployments/9075c49c-a3f1-419f-8073-18e2a2a728d2/online

### Test the API:

In [ ]:
# example input - row 25 (raw data)

example_values = [str(k) for k in X_test.iloc[25,:].values]
scoring_payload = {"fields": list(X_test.columns),"values":[example_values]}
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print(predictions)

# example output:
# {'fields': ['prediction', 'probability'], 'values': [[0, [0.5862835331496381, 0.41371646685036184]]]}

### Delete deployments if necessary

In [ ]:
client.deployments.list()

In [ ]:
client.repository.delete('- uid -')